In [1]:
# Imports:
import pandas as pd

In [2]:
# Constants:
EXAMPLE_URL = "https://en.wiktionary.org/wiki/%D0%BD%D0%B0%D0%B9%D1%82%D0%B8"

In [5]:
df1 = pd.read_html(EXAMPLE_URL)

In [6]:
df1.shape

AttributeError: 'list' object has no attribute 'shape'

In [7]:
df1.columns

AttributeError: 'list' object has no attribute 'columns'

In [8]:
len(df1)

3

In [9]:
df1[0]

,0,1,2
0,Audio:,NaN,(file)


In [10]:
df1[1]

,perfective aspect,perfective aspect.1,perfective aspect.2
0,infinitive,найти́ najtí,найти́ najtí
1,participles,present tense,past tense
2,active,—,наше́дший našédšij
3,passive,—,на́йденный nájdennyj
4,adverbial,—,"найдя́ najdjá, наше́дши našédši"
5,NaN,present tense,future tense
6,1st singular (я),—,найду́ najdú
7,2nd singular (ты),—,найдёшь najdjóšʹ
8,3rd singular (он/она́/оно́),—,найдёт najdjót
9,1st plural (мы),—,найдём najdjóm


In [11]:
# Attempt to scrape with better parsing (e.g. same term, will be common for many languages; English & German, Portuguese & Spanish, Russian & Ukrainian, etc.)
df2 = pd.read_html("https://en.wiktionary.org/wiki/%D1%8F#Ukrainian")
print(type(df2))

<class 'list'>


In [12]:
len(df2)

16

In [13]:
df2[0].shape

(5, 3)

In [14]:
df2[0]

,0,1,2
0,NaN,NaN,NaN
1,"я U+044F, &#1103; CYRILLIC SMALL LETTER YA ← ю...","я U+044F, &#1103; CYRILLIC SMALL LETTER YA ← ю...",NaN
2,я,"U+044F, &#1103; CYRILLIC SMALL LETTER YA",NaN
3,NaN,NaN,NaN
4,← ю [U+044E],Cyrillic,ѐ → [U+0450]


In [17]:
# Some inspection shows that we're likely looking for a table whose class name includes the word "inflection"
df3 = pd.read_html("https://en.wiktionary.org/wiki/%D1%8F#Ukrainian", attrs={"class": "inflection-table"})
print(type(df3))

<class 'list'>


In [18]:
df3[0].shape

(999, 12)

In [19]:
df3[0].head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,singular,singular,singular,singular,singular,NaN,plural,plural,plural,NaN,reflexive
1,NaN,1st person,2nd person,3rd person,3rd person,3rd person,NaN,1st person,2nd person,3rd person,NaN,reflexive
2,NaN,1st person,2nd person,m,n,f,NaN,1st person,2nd person,3rd person,NaN,reflexive
3,nominative,я (ja),ты (ty),ён (jon),яно́ (janó),яна́ (janá),NaN,мы (my),вы (vy),яны́ (janý),NaN,—
4,genitive,мяне́ (mjanjé),цябе́ (cjabjé),яго́ (jahó),яго́ (jahó),яе́ (jajé),NaN,нас (nas),вас (vas),іх (ix),NaN,сябе́ (sjabjé)


In [35]:
# trying with lxml so we can grab our preferred language and have some more control over what we extract from the tree.
try:
    from lxml import etree
    print("running with lxml.etree")
except ImportError:
    import xml.etree.ElementTree as etree
    print("running with Python's xml.etree.ElementTree")

running with lxml.etree


In [21]:
import requests

In [22]:
result = requests.get("https://en.wiktionary.org/api/rest_v1/page/title/Earth")

In [23]:
result

<Response [200]>

In [25]:
result = requests.get("https://en.wiktionary.org/api/rest_v1/page/title/я")

In [30]:
r = requests.get("https://en.wiktionary.org/api/rest_v1/page/html/%D1%8F")
r

<Response [200]>

In [34]:
len(r.text)

797397

In [36]:
et = etree.ElementTree(r.text)

TypeError: Argument 'element' has incorrect type (expected lxml.etree._Element, got str)

In [38]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.content, "lxml")
inflection_tables = soup.find_all("table", class_=lambda c: c and "inflection" in c)

In [39]:
type(inflection_tables)

bs4.element.ResultSet

In [40]:
len(inflection_tables)

9

In [42]:
type(inflection_tables[0])

bs4.element.Tag

In [44]:
dfs = [pd.read_html(str(table))[0] for table in inflection_tables]

/var/folders/z_/8kb9ps0s23lghzs8s5zd78br0000gn/T/ipykernel_5166/3212884710.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = [pd.read_html(str(table))[0] for table in inflection_tables]
/var/folders/z_/8kb9ps0s23lghzs8s5zd78br0000gn/T/ipykernel_5166/3212884710.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = [pd.read_html(str(table))[0] for table in inflection_tables]
/var/folders/z_/8kb9ps0s23lghzs8s5zd78br0000gn/T/ipykernel_5166/3212884710.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = [pd.read_html(str(table))[0] for table in inflection_tables]
/var/folders/z_/8kb9ps0s23lghzs8s5zd78br000

In [45]:
dfs[0]

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,singular,singular,singular,singular,singular,NaN,plural,plural,plural,NaN,reflexive
1,NaN,1st person,2nd person,3rd person,3rd person,3rd person,NaN,1st person,2nd person,3rd person,NaN,reflexive
2,NaN,1st person,2nd person,m,n,f,NaN,1st person,2nd person,3rd person,NaN,reflexive
3,nominative,я (ja),ты (ty),ён (jon),яно́ (janó),яна́ (janá),NaN,мы (my),вы (vy),яны́ (janý),NaN,—
4,genitive,мяне́ (mjanjé),цябе́ (cjabjé),яго́ (jahó),яго́ (jahó),яе́ (jajé),NaN,нас (nas),вас (vas),іх (ix),NaN,сябе́ (sjabjé)
...,...,...,...,...,...,...,...,...,...,...,...,...
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
dfs[1]

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
0           singular       first person       first person   
1           singular      second person           informal   
2           singular      second person             formal   
3           singular       third person          masculine   
4           singular       third person           feminine   
5           singular       third person             neuter   
6             plural       first person       first person   
7             plural      second person           informal   
8             plural      second person             formal   
9             plural       third person       third person   

   nominative (subject) accusative (direct complement)           \
   nominative (subject)                           full    short   
0               аз (az)         мен (men) ме́не (méne)  ме (me)   
1               ти (ti)         теб (teb) те́бе (tébe)  те (te)   
2            Ви́е (Víe)                      Вас (Vas)  Ви (Vi)   
3             той (toj)                   не́го (négo)  го (go)   
4              тя (tja)                    не́я (néja)   я (ja)   
5               то (to)                   не́го (négo)  го (go)   
6  ни́е (níe) ний (nij)                      нас (nas)  ни (ni)   
7  ви́е (víe) вий (vij)                      вас (vas)  ви (vi)   
8            Ви́е (Víe)                      Вас (Vas)  Ви (Vi)   
9               те (te)                     тях (tjah)  ги (gi)   

  dative (indirect complement)                    prepositional  
                          full    short           prepositional  
0                 ме́не (méne)  ми (mi)  мен (men) ме́не (méne)  
1                 те́бе (tébe)  ти (ti)  теб (teb) те́бе (tébe)  
2                    Вам (Vam)  Ви (Vi)               Вас (Vas)  
3                 не́му (nému)  му (mu)            не́го (négo)  
4                    ней (nej)    ѝ (ì)             не́я (néja)  
5                 не́му (nému)  му (mu)            не́го (négo)  
6                    нам (nam)  ни (ni)               нас (nas)  
7                    вам (vam)  ви (vi)               вас (vas)  
8                    Вам (Vam)  Ви (Vi)               Вас (Vas)  
9                   тям (tjam)  им (im)              тях (tjah)

In [47]:
dfs[2]

,0,1
0,Arabic,يا
1,Cyrillic,я
2,Latin,ia
3,Yañalif,ə


In [48]:
dfs[3]

Unnamed: 0_level_0  \
                                   Unnamed: 0_level_1   
                                   Unnamed: 0_level_2   
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                          nominative   
7                                            genitive   
8                                              dative   
9                                          accusative   
10                                     instrumental 5   
11                                         locative 5   
12  Used with reflexive verbs. The alternative for...   

                                             singular  \
                                           1st person   
                                           1st person   
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                              я (ja)   
7                               мнє, ме 1 ‎(mnje, me)   
8                                  мнє, ми (mnje, mi)   
9                                 мнє, ме ‎(mnje, me)   
10                                          мну (mnu)   
11                                        мнє ‎(mnje)   
12  Used with reflexive verbs. The alternative for...   

                                                       \
                                           2nd person   
                                           2nd person   
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                             ти (ti)   
7                              тебе, це 1 ‎(tebe, ce)   
8                                тебе, ци ‎(tebe, ci)   
9                                тебе, це ‎(tebe, ce)   
10                                       тобу ‎(tobu)   
11                               тебе, це ‎(tebe, ce)   
12  Used with reflexive verbs. The alternative for...   

                                                       \
                                           3rd person   
                                                    m   
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                           вон (von)   
7   його 1, нього 2, ньго 2, го 1 (joho, nʹoho, nʹ...   
8              йому 4, ньому 2, му ‎(jomu, nʹomu, mu)   
9   його 4, нього 2, ньго 2, го (joho, nʹoho, nʹho...   
10                                         нїм (njim)   
11                         нїм / ньому (njim / n'omu)   
12  Used with reflexive verbs. The alternative for...   

                                                       \
                                                        
                                                    n   
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                        

In [49]:
dfs[4]

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,singular,singular,singular,singular,singular,NaN,plural,plural,plural,NaN,reflexive
1,NaN,1st person,2nd person,3rd person,3rd person,3rd person,NaN,1st person,2nd person,3rd person,NaN,reflexive
2,NaN,1st person,2nd person,m,n,f,NaN,1st person,2nd person,3rd person,NaN,reflexive
3,nominative,я (ja),ты (ty),он (on),оно́ (onó),она́ (oná),NaN,мы (my),вы (vy),они́ 1 (oní),NaN,—
4,genitive,меня́ (menjá),тебя́ (tebjá),"его́ 2, него́ 2 3 (jevó, nevó)","его́ 2, него́ 2 3 (jevó, nevó)","её 4, неё 3 4 (jejó, nejó)",NaN,нас (nas),вас (vas),"их, них 3 (ix, nix)",NaN,себя́ (sebjá)
5,dative,мне (mne),тебе́ (tebé),"ему́, нему́ 3 (jemú, nemú)","ему́, нему́ 3 (jemú, nemú)","ей, ней 3 (jej, nej)",NaN,нам (nam),вам (vam),"им, ним 3 (im, nim)",NaN,себе́ (sebé)
6,accusative,меня́ (menjá),тебя́ (tebjá),"его́ 2, него́ 2 3 (jevó, nevó)","его́ 2, него́ 2 3 (jevó, nevó)","её, неё 3 (jejó, nejó)",NaN,нас (nas),вас (vas),"их, них 3 (ix, nix)",NaN,себя́ (sebjá)
7,instrumental,"мной, мно́ю 5 (mnoj, mnóju)","тобо́й, тобо́ю 5 (tobój, tobóju)","им, ним 3 (im, nim)","им, ним 3 (im, nim)","ей, ней 3, е́ю 5, не́ю 3 5 (jej, nej, jéju, néju)",NaN,на́ми (námi),ва́ми (vámi),"и́ми, ни́ми 3 (ími, ními)",NaN,"собо́й, собо́ю 5 (sobój, sobóju)"
8,prepositional 6,мне (mne),тебе́ (tebé),нём 3 (njom),нём 3 (njom),ней 3 (nej),NaN,нас (nas),вас (vas),них 3 (nix),NaN,себе́ (sebé)


In [50]:
rus_inflections = dfs[4]

In [53]:
rus_inflections = rus_inflections.dropna(how="all")

In [54]:
rus_inflections.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')

In [55]:
rus_inflections.iloc[:, 0]

0                NaN
1                NaN
2                NaN
3         nominative
4           genitive
5             dative
6         accusative
7       instrumental
8    prepositional 6
Name: 0, dtype: object

In [58]:
rus_inflections[rus_inflections.iloc[:, 0].astype(str).str.lower().str.contains("nominative")]

,0,1,2,3,4,5,6,7,8,9,10,11
3,nominative,я (ja),ты (ty),он (on),оно́ (onó),она́ (oná),NaN,мы (my),вы (vy),они́ 1 (oní),NaN,—


In [65]:
def extract_case(inflections_df: pd.DataFrame, case: str) -> pd.DataFrame:
    """Given a table of inflections for a noun or adjective, over all 
    cases and persons, extract the inflections for each person in that
    case. e.g. if given an inflection table for Ukrainian and a case
    value of "nominative," returns a one-row `DataFrame` with all
    nominative case forms of the noun/adjective.
    """
    # Cleanup, maybe best to move to a separate function called earlier
    # at some point in the future.
    inflections_df = inflections_df.dropna(how="all")
    boolean_indexer = inflections_df.iloc[:, 0] \
        .astype(str).str.lower().str.contains(case.lower())
    declensions = inflections_df[boolean_indexer]
    return declensions

In [69]:
genitive_pronouns = extract_case(rus_inflections, "genitive")
genitive_pronouns

,0,1,2,3,4,5,6,7,8,9,10,11
4,genitive,меня́ (menjá),тебя́ (tebjá),"его́ 2, него́ 2 3 (jevó, nevó)","его́ 2, него́ 2 3 (jevó, nevó)","её 4, неё 3 4 (jejó, nejó)",NaN,нас (nas),вас (vas),"их, них 3 (ix, nix)",NaN,себя́ (sebjá)


In [67]:
type(genitive_pronouns)

pandas.core.frame.DataFrame

In [71]:
genitive_pronouns.values.flatten()

array(['genitive', 'меня́ (menjá)', 'тебя́ (tebjá)',
       'его́\u202f2, него́\u202f2\xa03 (jevó, nevó)',
       'его́\u202f2, него́\u202f2\xa03 (jevó, nevó)',
       'её\u202f4, неё\u202f3\xa04 (jejó, nejó)', nan, 'нас (nas)',
       'вас (vas)', 'их, них\u202f3 (ix, nix)', nan, 'себя́ (sebjá)'],
      dtype=object)

In [72]:
genitive_pronouns.stack()

4  0                           genitive
   1                      меня́ (menjá)
   2                      тебя́ (tebjá)
   3     его́ 2, него́ 2 3 (jevó, nevó)
   4     его́ 2, него́ 2 3 (jevó, nevó)
   5         её 4, неё 3 4 (jejó, nejó)
   7                          нас (nas)
   8                          вас (vas)
   9                их, них 3 (ix, nix)
   11                     себя́ (sebjá)
dtype: object

In [73]:
genitive_pronouns.stack().dropna()

4  0                           genitive
   1                      меня́ (menjá)
   2                      тебя́ (tebjá)
   3     его́ 2, него́ 2 3 (jevó, nevó)
   4     его́ 2, него́ 2 3 (jevó, nevó)
   5         её 4, неё 3 4 (jejó, nejó)
   7                          нас (nas)
   8                          вас (vas)
   9                их, них 3 (ix, nix)
   11                     себя́ (sebjá)
dtype: object

In [78]:
import re

PARENTHESIZE = re.compile("""\(.*\)""")

<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
/var/folders/z_/8kb9ps0s23lghzs8s5zd78br0000gn/T/ipykernel_5166/676826682.py:3: SyntaxWarning: invalid escape sequence '\('
  PARENTHESIZE = re.compile("""\(.*\)""")


In [79]:
re.sub(PARENTHESIZE, "", "их, них 3 (ix, nix)")

'их, них\u202f3 '

In [96]:
def extract_case(inflections_df: pd.DataFrame, case: str) -> pd.DataFrame:
    """Given a table of inflections for a noun or adjective, over all 
    cases and persons, extract the inflections for each person in that
    case. e.g. if given an inflection table for Ukrainian and a case
    value of "nominative," returns a one-row `DataFrame` with all
    nominative case forms of the noun/adjective.
    """
    # Cleanup, maybe best to move to a separate function called earlier
    # at some point in the future.
    # Remove rows that are all NaN
    inflections_df = inflections_df.dropna(how="all")

    # Filter for the target case in the first column
    boolean_indexer = inflections_df.iloc[:, 0] \
        .astype(str).str.lower().str.contains(case.lower())
    declensions = inflections_df[boolean_indexer]

    # Remove any parenthesized content (e.g., pronunciations like "(jevó)")
    declensions = declensions.map(
        lambda x: re.sub(r"\s*\([^)]*\)", "", str(x)) if pd.notna(x) else x
    )

    return declensions

In [97]:
genitive_pronouns_2 = extract_case(rus_inflections, "genitive")
genitive_pronouns_2

,0,1,2,3,4,5,6,7,8,9,10,11
4,genitive,меня́,тебя́,"его́ 2, него́ 2 3","его́ 2, него́ 2 3","её 4, неё 3 4",NaN,нас,вас,"их, них 3",NaN,себя́


In [98]:
r.raise_for_status()